In [47]:
import os
import pandas as pd
import glob
import pickle
from torch.utils.data import DataLoader, Dataset
import numpy as np
from osgeo import gdal

In [48]:
# Change server_mount for your system
server_mount = '/home/blair/server/herd_hover' 

# Specify radius of viewshed (in meters)
viewshed_radius = 100
# Specify height/width of downsampled viewshed, e.g. 512 will return an array of 512x512 pixels
viewshed_hw = 512
# Specify radius (meters) used to define social density (number of conspecifics within this radius = social density)
social_radius = 10

data_folder = os.path.join(server_mount, 'zebra_movement_data')
steps_dir = os.path.join(data_folder, 'steps_5m')
social_data_dir = os.path.join(data_folder, 'social_data')
obs_metadata_file = os.path.join(data_folder, 'observation_metadata.csv')
track_metadata_file = os.path.join(data_folder, 'track_metadata.csv')
#ob_dist_dir = os.path.join(data_folder, 'observer_distances')
rasters_dir = os.path.join(data_folder, 'rasters')

In [49]:
# Define the function to generate the viewshed
def generate_viewshed(dsm_file, X, Y, height, targetRasterName, radius):
    src_ds = gdal.Open(dsm_file)      
    srcBand = src_ds.GetRasterBand(1)
    #c_options = ['COMPRESS=LZW', 'NUM_THREADS=4']
    c_options = ['NUM_THREADS=4']
    
    gdal.ViewshedGenerate(
        srcBand=srcBand,
        driverName="GTIFF",
        targetRasterName=targetRasterName,
        creationOptions=c_options,
        observerX=X,
        observerY=Y,
        observerHeight=height,
        targetHeight=0,
        visibleVal=1,
        invisibleVal=0,
        outOfRangeVal=-10000,
        noDataVal=-10000,
        dfCurvCoeff=0.85714,
        mode=1,
        maxDistance=radius
    )
    src_ds = None

In [66]:
class ZebraMovementDataset(Dataset):
    def __init__(self, steps_dir, obs_metadata_file, track_metadata_file, rasters_dir, viewshed_radius, viewshed_hw, social_data_dir, social_radius):
        step_files = glob.glob(os.path.join(steps_dir, "*.pkl"))
        self.steps = pd.concat((pd.read_pickle(f) for f in step_files), ignore_index = True)
        self.obs_metadata = pd.read_csv(obs_metadata_file)
        self.track_metadata = pd.read_csv(track_metadata_file)
        self.viewshed_radius = viewshed_radius
        self.viewshed_hw = viewshed_hw
        self.social_data_dir = social_data_dir
        self.social_radius = social_radius

    def __n_obs__(self): ## gives the number of observations in the dataset
        return len(self.obs_metadata)

    ### The following two functions are inaccurate because the track metadata file includes
    ### lines for tracks that are filtered out because they are too short (no steps or only 1 step)
    # def __n_tracks__(self): ## gives the number of tracks in the dataset
    #     return len(self.track_metadata)

    # def __n_animals__(self): ## gives the number of unique animals in the dataset
    #     return len(self.track_metadata.individual_ID.unique())

    def __len__(self): ## here somehow need to get the total number of observed + inferred steps
        return(len(self.steps))

    def __getitem__(self, idx):
        step_id = self.steps.loc[idx, "id"] # unique ID for each step
        frame = self.steps.loc[idx, "frame"] # video frame (time step) corresponding to step location
        
        # METADATA
        ob_num = step_id.split('_')[0].split('b')[1]
        observation = "observation" + step_id.split('_')[0].split('b')[1] # observation number
        track_num = int(step_id.split('_')[1].split('k')[1]) 
        track = observation + '-track' + "{:02d}".format(track_num) # track id ('ob000-track00')
        animal_id = self.track_metadata[(self.track_metadata['observation'] == observation) & (self.track_metadata['track'] == track_num)]['individual_ID'].item() # animal id (some animals have multiple tracks)
        species = self.track_metadata[(self.track_metadata['observation'] == observation) & (self.track_metadata['track'] == track_num)]['species'].item() # currently either pz (plains zebra) or gz (grevy's zebra)
        age_class = self.track_metadata[(self.track_metadata['observation'] == observation) & (self.track_metadata['track'] == track_num)]['age'].item() # either "adult" or "young"
        scare = True if frame >= self.obs_metadata[self.obs_metadata['observation'] == observation]['scare_frame'].item() else False # has a scare happened before this frame?
        
        # STEP INFO
        step_type = 'observed' if step_id.split('_')[-1] == 'ob' else 'simulated' # observed (real steps) or simulated (fake steps - 5 simulated per real step)
        if self.steps[self.steps['id'] == step_id]['prev_step'].item() == None:
            prev_step_id = "None"
        else:
            prev_step_id = self.steps[self.steps['id'] == step_id]['prev_step'].item()# id of previous observed step
        
        step_length_m = self.steps[self.steps['id'] == step_id]['step_length_m'].item() # length of step (should be approximately 5m)
        step_duration_s = self.steps[self.steps['id'] == step_id]['step_duration_s'].item() # time duration of step in seconds
        step_speed_mps = self.steps[self.steps['id'] == step_id]['step_speed_mps'].item() # speed of step in meters per second
        
        # SPATIAL RELATIONSHIP TO OBSERVATION FIELD TEAM
        ob_dist = self.steps[self.steps['id'] == step_id]['dist_to_observer'].item() # absolute distance between the step location and the stationary observation team (in meters)
        delta_ob_dist = self.steps[self.steps['id'] == step_id]['delta_observer_dist'].item() # change in distance to the observation team since the last observed step location: negative values mean animal has moved closer
        ob_angle = self.steps[self.steps['id'] == step_id]['angle_to_observers'].item() # angle of step relative to vector toward observation team - 0 means moving directly toward observation team, 180 means moving directly away.

        # VIEWSHED CALCULATION
        # observer heights are stored in self.steps in column observer_height
        # dsm rasters are in rasters_dir/DSMs
        X = self.steps.loc[idx, 'lon']
        Y = self.steps.loc[idx, 'lat']
        height = float(self.steps.loc[idx, 'observer_height'])
        map_name = self.obs_metadata[self.obs_metadata['observation'] == observation]['big_map'].item() + '_dsm.tif'
        dsm = os.path.join(rasters_dir, 'DSMs', map_name)
        # save the raster in memory
        full_raster_name = step_id + '_tempraster.tif'
        full_raster = '/vsimem/' + full_raster_name
        # generate the full-resolution viewshed
        generate_viewshed(dsm, X, Y, height, full_raster, self.viewshed_radius)
        # load the full-resolution viewshed and calculate the proportion of pixels that are visible
        vs = gdal.OpenEx(full_raster)
        vshed = vs.GetRasterBand(1)
        mean_full = vshed.GetStatistics(0, 1)[2]
        # downsample the raster to the specified dimensions
        downsample_raster_name = step_id + 'tempraster2.tif'
        downsample_raster = '/vsimem/' + downsample_raster_name
        
        kws = gdal.WarpOptions(
            format = 'GTiff',
            width = self.viewshed_hw,
            height = self.viewshed_hw,
            srcBands = [1],
            resampleAlg = 'average', # each pixel in the downsampled raster is the mean of the pixels it overlaps in the full-res raster
            outputType = gdal.GDT_Float64
        )
        gdal.Warp(
            destNameOrDestDS=downsample_raster,
            srcDSOrSrcDSTab=vs, options=kws)
        
        vs = None
        gdal.Unlink(full_raster)
        # convert downsampled raster into numpy array
        mvs = gdal.OpenEx(downsample_raster)
        viewshed_array = mvs.ReadAsArray(buf_type=gdal.GDT_Float64)
        mvs = None
        viewshed_array[viewshed_array==5]=np.nan # "no data" value in the original viewshed is 5, replace it here with np.nan
        gdal.Unlink(downsample_raster)
        
        # scarers_dist = STILL NEED TO ADD - How far is this location from the scarers?
        # scarers_ang = STILL NEED TO ADD - What is the angle of the step relative to the direction to the scarers?
        # delta_scarers_dist = STILL NEED TO ADD - How has distance to the scarers changed with this step?

        # HABITAT VARIABLES
        slope = self.steps.loc[idx, 'ground_slope'] # angle of incline of step from last point to current point
        road = self.steps.loc[idx, 'road'] # is the point on a road (1) or not (0)?
        # trail = Boolean, is the point on a trail?

        # SOCIAL VARIABLES
        social_data_file = os.path.join(self.social_data_dir, 'ob%s_track%s_socialdata.pkl' %(ob_num, "{:02d}".format(track_num)))
        with open(social_data_file, 'rb') as f:
            social_data = pickle.load(f)
        focal_dat = [d for d in social_data if d['step_id'] == step_id][0]
        social = pd.DataFrame.from_dict(focal_dat)
        nn_dist = min(social[social['neighbor_spps'] == species]['neighbor_distances']) if len(social[social['neighbor_spps'] == species]) >0 else np.nan # distance to nearest conspecific neighbor
        nn_dist_hetero = min(social[social['neighbor_spps'] != species]['neighbor_distances']) if len(social[social['neighbor_spps'] != species]) >0 else np.nan # distance to nearest heterospecific neighbor
        soc_dens = len(social[(social['neighbor_spps'] == species) & (social['neighbor_distances'] <= self.social_radius)]) # social density, conspecifics only
        tot_dens = len(social[social['neighbor_distances'] <= self.social_radius]) # social density, all species
        prop_vis = len(social[(social['neighbor_spps'] == species) & (social['neighbor_visibility'] == True)])/(len(social[(social['neighbor_spps'] == species) & (social['neighbor_visibility'] == True)]) + len(social[(social['neighbor_spps'] == species) & (social['neighbor_visibility'] == False)])) # proportion of conspecifics in social radius
        prop_vis_tot = len(social[social['neighbor_visibility'] == True])/(len(social[social['neighbor_visibility'] == True]) + len(social[social['neighbor_visibility'] == False])) # proportion of all animals in social radius
        # used = used space - how many conspecifics have used this space (within 2m buffer) so far in observation?
        
        return step_id, step_type, observation, track, animal_id, species, age_class, step_length_m, step_duration_s, step_speed_mps, ob_dist, delta_ob_dist, ob_angle, scare, mean_full, viewshed_array, slope, road, nn_dist, nn_dist_hetero, soc_dens, tot_dens, prop_vis, prop_vis_tot

In [67]:
dataset = ZebraMovementDataset(steps_dir, 
                               obs_metadata_file, 
                               track_metadata_file, 
                               rasters_dir, 
                               viewshed_radius = viewshed_radius, 
                               viewshed_hw = viewshed_hw, 
                               social_data_dir = social_data_dir,
                               social_radius = social_radius)

In [68]:
dataloader = DataLoader(dataset, batch_size = 2, shuffle = True)

In [69]:
test = next(iter(dataloader))

In [70]:
test[:]

[('ob088_track13_f80503_sim-1', 'ob015_track07_f12587_sim-1'),
 ('simulated', 'simulated'),
 ('observation088', 'observation015'),
 ('observation088-track13', 'observation015-track07'),
 ('088-013', '015-007'),
 ('gz', 'gz'),
 ('young', 'adult'),
 tensor([5.0229, 5.0105], dtype=torch.float64),
 tensor([1.0667, 4.4667], dtype=torch.float64),
 tensor([4.7090, 1.1218], dtype=torch.float64),
 tensor([609.1954, 308.2825], dtype=torch.float64),
 tensor([4.3583, 0.6372], dtype=torch.float64),
 tensor([150.0730,  96.8441], dtype=torch.float64),
 tensor([ True, False]),
 tensor([0.0859, 0.0299], dtype=torch.float64),
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0